In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
import pandas as pd
test=pd.read_csv("/kaggle/input/data-science-bowl-2019/test.csv")
test.sample(5)

In [ ]:
import json
test_sample=test.loc[test["installation_id"]=="163ffbd7"]
test_sample_assessment=test_sample.loc[test_sample["type"]=="Assessment"]
print(test_sample_assessment.world.unique())
print(test_sample_assessment.title.unique())
print(test_sample_assessment.game_session.unique())
#All the assessment game session in this specific installation_id#


In [ ]:
test_sample_game_sample=test_sample_assessment.loc[test_sample_assessment["game_session"]=="8c4fb55e2605b2db"]
for i in test_sample_game_sample.event_data:
    print(i)
    
#This last game is the one that need to be predicted#
#so the features need to be engineered in such way that it resembles the information avaivable to the test set#
#seems like the training set can be constructed in such way that the last game session in the title is held out as the ground truth case for prediction

# Below trys to find all the game session associated with this installation_id

# ??? use one installation_id to construct one row of feature table ?

In [ ]:

print(test_sample.world.unique())
print(test_sample.title.unique())
print(test_sample.game_session.unique())

In [ ]:
game_session_id=test_sample.game_session.unique()
test_features=pd.DataFrame(game_session_id,columns=["game_session"])


test_features["title"]=np.zeros(len(game_session_id))
test_features["start_game"]=np.zeros(len(game_session_id))
test_features["end_game"]=np.zeros(len(game_session_id))
test_features["duration"]=np.zeros(len(game_session_id))
test_features["type"]=np.zeros(len(game_session_id))
test_features["type"]=np.zeros(len(game_session_id))
test_features["world"]=np.zeros(len(game_session_id))
count=0
for i in game_session_id:
    current_game_events=test_sample.loc[test_sample["game_session"]==i]
    test_features["title"].iloc[count]=current_game_events.title.iloc[0]
    test_features["start_game"].iloc[count]=current_game_events.timestamp.iloc[0]
    test_features["end_game"].iloc[count]=current_game_events.timestamp.iloc[-1]
    test_features["duration"].iloc[count]=(pd.to_datetime(current_game_events.timestamp.iloc[-1])-pd.to_datetime(current_game_events.timestamp.iloc[0])).seconds*1000
    test_features["type"].iloc[count]=current_game_events.type.iloc[0]
    test_features["world"].iloc[count]=current_game_events.world.iloc[0]
    count+=1
    
    
test_features

# Out of all the game sessions, check assessment session accuracy score

In [ ]:
import json
# test_features_valid=test_features.loc[test_features["type"]=="Assessment"].game_session
test_features_valid=test_features.game_session

for i in test_features_valid:
    if(i=="2b2ebef9dad7349e"):
        events=test_sample.loc[test_sample["game_session"]==i].event_data
        for e in events:
            print(json.loads(e))

test_features_valid


In [ ]:
## remember to filter the last one out
import json
def calculate_ground_truth(game_session):
    events=test_sample.loc[test_sample["game_session"]==game_session].event_data
    succeed=[]
    fail=[]
    for e in events:
        a=json.loads(e)
        event_code=int(a["event_code"])
        if((event_code==4100) or (event_code==4110)):
            correct=a["correct"]
            print(correct)
            if(correct==True):
                succeed.append(e)
                break
            elif(correct==False):
                fail.append(e)
                
    
    if(len(fail)==0):
        accuracy=1
    else:
        accuracy=len(succeed)/(len(fail)+len(succeed))

    if(len(succeed)==0):
        accuracy_group=0
    elif (len(fail)==1):
        accuracy_group=2
    elif (len(fail)>=2):
        accuracy_group=1
    elif (len(fail)==0):
        accuracy_group=3
    return len(succeed),len(fail),accuracy,accuracy_group

calculate_ground_truth("2b2ebef9dad7349e")

In [ ]:

count=0

# test_features_aseessment=test_features.loc[test_features["type"]=="Assessment"]
test_features_aseessment=test_features

test_features_aseessment["num_incorrect"]=np.zeros(len(test_features_aseessment))
test_features_aseessment["num_correct"]=np.zeros(len(test_features_aseessment))
test_features_aseessment["accuracy"]=np.zeros(len(test_features_aseessment))
test_features_aseessment["accuracy_group"]=np.zeros(len(test_features_aseessment))

for i in test_features_valid:
    num_correct,num_incorrect,accuracy,accuracy_group=calculate_ground_truth(i)
    test_features_aseessment["num_incorrect"].iloc[count]=num_incorrect
    test_features_aseessment["num_correct"].iloc[count]=num_correct
    test_features_aseessment["accuracy"].iloc[count]=accuracy
    test_features_aseessment["accuracy_group"].iloc[count]=accuracy_group

    count+=1
    
# test_features_aseessment.drop("number_correct",axis=1,inplace=True)
test_features_aseessment

In [ ]:
test_features_aseessment['Datetime'] = pd.to_datetime(test_features_aseessment["start_game"])
test_features_aseessment = test_features_aseessment.set_index('Datetime')
test_features_aseessment.drop(["start_game","end_game"],axis=1,inplace=True)
test_features_aseessment

In [ ]:
test_features_aseessment["Month"]=test_features_aseessment.index.month
test_features_aseessment["Year"]=test_features_aseessment.index.year
test_features_aseessment["Weekday"]=test_features_aseessment.index.dayofweek

test_features_aseessment


## Extracting more event data into each governing game row ? necessary ?
## Re-arrange the in-between-non-asssessment-game so that each row is a assessment with information about previous game.
## Is this a time series problem ?

In [ ]:
## below is to add the event info into each game, i.e. time in between each attempt. 
def calculate_attempt_inteval(game_session):
    game_time_holder=[]
    event_count_holder=[]
    game_time=[]
    event_count=[]
    events=test_sample.loc[test_sample["game_session"]==game_session].event_data
    for e in events:
        a=json.loads(e)
        event_code=int(a["event_code"])
        if((event_code==4100) or (event_code==4110)):
            game_time_holder.append(a["game_time"])
            event_count_holder.append(a["event_count"])
            
    for i in range(len(game_time_holder)):
        if i==0:
            game_time.append(game_time_holder[i])
        elif i<2:
            game_time.append(game_time_holder[i]-game_time_holder[i-1])
        elif i>=2:
            game_time.append(sum(game_time_holder)-game_time_holder[0]-game_time_holder[1]-game_time_holder[2])
            break
    
    for j in range(len(event_count_holder)):
        if j==0:
            event_count.append(event_count_holder[j])

        elif j<2:
            event_count.append(event_count_holder[j]-event_count_holder[j-1])
        elif j>=2:
            event_count.append(sum(event_count_holder)-event_count_holder[0]-event_count_holder[1]-event_count_holder[2])
            break
            
    return game_time,event_count
            

calculate_attempt_inteval("95dda796ba5fce1e")

In [ ]:
test_features_aseessment["1_attempt_time_interval"]=np.zeros(len(game_session_id))
test_features_aseessment["2_attempt_time_interval"]=np.zeros(len(game_session_id))
test_features_aseessment["3_attempt_time_interval"]=np.zeros(len(game_session_id))
test_features_aseessment["1_attempt_event_count"]=np.zeros(len(game_session_id))
test_features_aseessment["2_attempt_event_count"]=np.zeros(len(game_session_id))
test_features_aseessment["3_attempt_event_count"]=np.zeros(len(game_session_id))


for i in range(len(game_session_id)):
    game_time,event_count=calculate_attempt_inteval(game_session_id[i])

    if (len(game_time)==0):
        continue
    print(i,game_time,event_count)
    if(len(game_time)>=1):
        test_features_aseessment["1_attempt_time_interval"].iloc[i]=game_time[0]
        test_features_aseessment["1_attempt_event_count"].iloc[i]=event_count[0]
    if(len(game_time)>=2):
        test_features_aseessment["2_attempt_time_interval"].iloc[i]=game_time[1]
        test_features_aseessment["2_attempt_event_count"].iloc[i]=event_count[1]
    if(len(game_time)>=3):
        test_features_aseessment["3_attempt_event_count"].iloc[i]=event_count[2]
        test_features_aseessment["3_attempt_time_interval"].iloc[i]=game_time[2]

test_features_aseessment